<link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Sofia">
<style>
  @import url('https://fonts.googleapis.com/css2?family=Concert+One&display=swap');
  .topic {
    text-align: center;
    font-weight: 900;
    font-size: 50px;
    text-shadow: 3px 3px 3px #ababab;
    font-family: 'Concert One', sans-serif;
  }
  @import url('https://fonts.googleapis.com/css2?family=Lobster&display=swap');
  .info {
    text-align: center;
    font-weight: 800;
    font-size: 40px;
    font-family: 'Lobster', sans-serif;
  }
</style>

<h1 class=topic>Google Analytics Customer Revenue Prediction</h1>
<h2 class=info>Vũ Minh Phát - 21127739</h2>

___

## **<u>Các thư viện cần sử dụng</u>**

In [1]:
import numpy as np
import pandas as pd
# Seaborn and matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
# Plotly
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
import plotly.express as px

init_notebook_mode(connected=True)

---

## **<u>Đọc dữ liệu đã được làm sạch từ file</u>**

In [2]:
cleaned_df = pd.read_csv("../data/processed/train_preprocessed.csv")

In [3]:
# TEST
cleaned_df.sample(n=10)

,channelGrouping,date,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,...,totals.newVisits,totals.transactionRevenue,trafficSource.source,trafficSource.medium,trafficSource.isTrueDirect,date.day,date.year,date.month,date.weekDay,date.weekOfYear
705977,Organic Search,2016-11-19,1,Chrome,Windows,False,desktop,Americas,Northern America,United States,...,True,0.0,google,organic,False,19,2016,11,5,46
213018,Direct,2016-08-03,4,Chrome,Android,True,mobile,Americas,Northern America,United States,...,False,0.0,direct,direct,True,3,2016,8,2,31
287792,Paid Search,2017-07-28,1,Safari,iOS,True,mobile,Americas,Northern America,United States,...,True,0.0,google,cpc,False,28,2017,7,4,30
861896,Referral,2017-02-12,1,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,...,True,0.0,others,referral,False,12,2017,2,6,6
843212,Organic Search,2017-04-07,2,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,...,False,16990000.0,google,organic,True,7,2017,4,4,14
835913,Referral,2017-07-25,1,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,...,True,0.0,googleplex,referral,False,25,2017,7,1,30
319679,Social,2017-03-30,1,Chrome,Windows,False,desktop,Asia,Southern Asia,India,...,True,0.0,quora,referral,False,30,2017,3,3,13
453766,Organic Search,2017-07-03,1,Chrome,Windows,False,desktop,Europe,Western Europe,Germany,...,True,0.0,google,organic,False,3,2017,7,0,27
199096,Direct,2016-09-15,1,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,...,True,0.0,direct,direct,True,15,2016,9,3,37
103539,Referral,2017-07-04,1,Chrome,Macintosh,False,desktop,Oceania,Australasia,Australia,...,True,0.0,google,referral,False,4,2017,7,1,27


In [4]:
# TEST
cleaned_df.shape

(903653, 25)

---

# TODO: Mỗi câu hỏi sẽ phân tích số lượng truy cập + tổng tiền giao dịch

## Đưa ra câu hỏi cần trả lời

Sau khi có cái nhìn tổng quát về dữ liệu, ta thấy có một số vấn đề cần được quan tâm:

**Câu hỏi N-th**: Vào khoảng thời gian nào thì người dùng có xu hướng truy cập thường xuyên và sẵn sàng chi tiêu nhiều hơn để mua sắm các sản phẩm từ cửa hàng của Google?

**Trả lời câu hỏi này sẽ giúp chúng ta**: hiểu rõ hơn về xu hướng chi tiêu của người dùng và những thời điểm tiềm năng mà người dùng có khả năng chi tiếu nhiều hơn để mua sản phẩm từ cửa hàng. Đây là thông tin vô cùng cần thiết giúp các chiến dịch marketing dễ dàng tiếp cận và thu hút được sự chú ý từ người dùng, nâng cao khả năng tăng doanh thu cho cửa hàng.

---

## Phân tích dữ liệu để trả lời từng câu hỏi

## Tiền xử lý + phân tích dữ liệu để trả lời cho câu hỏi N-th

Đầu tiên, ta xây dựng một hàm tiện ích giúp đơn giản quá trình thống kê dựa trên thuộc tính mà ta quan tâm

In [5]:
def _create_feature_stats(df: pd.DataFrame, feature: str) -> pd.DataFrame:
    """Create a dataframe contains: 
        1. Number of visits per unique value in `feature`,
        2. Number of non-zero revenue visits per unique value in `feature` 
        3. Total revenue per unique value in `feature`.
    """
    # Create 3 dataframes
    counts = df[feature].value_counts().to_frame().reset_index()\
                .sort_values(feature).rename(columns={'count': 'visits'})

    non_zero_counts = \
        df.query("`totals.transactionRevenue` > 0")[feature]\
            .value_counts().to_frame().reset_index()\
                .sort_values(feature).rename(columns={'count': 'non_zero_transactions'})
    
    sums = df.groupby(feature)['totals.transactionRevenue'].sum()\
                .to_frame().reset_index().sort_values(feature)

    # Merge 3 dataframes
    merged_df = pd.merge(counts, non_zero_counts, on=feature, how='left')
    merged_df = pd.merge(merged_df, sums, on=feature, how='left')
    
    return merged_df

#### 1. Phân bố theo `ngày trong tháng`

In [6]:
# Get some useful statistics from `date.day` column
day_stats = _create_feature_stats(cleaned_df, 'date.day')

# Plot bar charts
BASIC_PLOTLY_CONFIG = dict(
    title_x=0.5, title_font_size=20,
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),
    coloraxis_showscale=False,
    xaxis_title_font_size=20, xaxis_tickfont_size=16,
    yaxis_title_font_size=20, yaxis_tickfont_size=16,
)
# ====================  For `visits` ==================== 
fig = px.bar(
    day_stats, x="date.day", y="visits",
    color='visits', hover_name='visits',
    labels={'date.day': 'Day', 'visits': '#Visits'},
    color_continuous_scale=px.colors.sequential.Tealgrn,
    height=400, width=800
)
fig.layout.update(
    BASIC_PLOTLY_CONFIG |
    dict(title_text='<b>Visits by day</b>', 
         xaxis_title='Day of month', yaxis_title='# Visits')
)
iplot(fig)

# ====================  For `non_zero_transactions` ====================
fig = px.bar(
    day_stats, x="date.day", y="non_zero_transactions",
    color='non_zero_transactions', hover_name='non_zero_transactions',
    labels={'date.day':'Day', 'non_zero_transactions':'#Non-zero transactions'},
    color_continuous_scale=px.colors.sequential.Blues,
    height=400, width=800
)
fig.layout.update(
    BASIC_PLOTLY_CONFIG |
    dict(title_text='<b>Non-zero transactions by day</b>', 
         xaxis_title='Day of month', yaxis_title='# Non-zero transactions')
)
iplot(fig)

# ====================  For `totals.transactionRevenue` ====================
fig = px.bar(
    day_stats, x="date.day", y="totals.transactionRevenue",
    color='totals.transactionRevenue', hover_name='totals.transactionRevenue',
    labels={'date.day': 'Day', 'totals.transactionRevenue': 'ΣRevenue'},
    color_continuous_scale=px.colors.sequential.Pinkyl,
    height=400, width=800
)
fig.layout.update(
    BASIC_PLOTLY_CONFIG |
    dict(title_text='<b>Transaction revenue by day</b>', 
         xaxis_title='Day of month', yaxis_title='Σ Revenue')
)
# iplot(fig)
fig.show()

> 💭 <u>**Nhận xét**</u>: 
> * Về tổng số lượt truy cập: Nhìn chung, tổng số lượng truy cập của người dùng vào GStore là không quá khác biệt giữa các ngày trong tháng. Ta thấy ngày 31 có số lượng truy cập thấp hơn hẳn những ngày khác, điều này cũng bình thường vì 1 năm chỉ có 7 tháng là có ngày thứ 31.
>
> * Về số lượt truy cập có giao dịch: Có vẻ như người dùng thường có tỉ lệ chi trả nhiều hơn vào các ngày giữa tháng (khoảng từ ngày 5 $\to$ 20 hàng tháng). Ngược lại, khả năng một người dùng chịu chi trả tiền vào cuối tháng thường tương đối thấp.
>
> * Về doanh thu theo từng ngày trong tháng: Cũng không ngoài dự đoán, các ngày có tỉ lệ người dùng chi trả cao hơn thì cũng có tổng doanh thu cao hơn. Tổng doanh thu của cửa hàng thường tập trung từ ngày 5 $\to$ 20 hàng tháng. Giai đoạn chuyển tiếp từ cuối tháng trước đến đầu tháng sau thường có tổng doanh thu không cao.
>
> * Tuy nhiên, các ngày trong tháng không mang tính phổ quát quá cao nên ta cần quan sát thêm các thông tin khác để có thể nắm bắt thói quen của khách hàng.

#### 2. Phân bố theo `ngày trong tuần`

In [7]:
# Get some useful statistics from `date.weekDay` column
weekDay_stats = _create_feature_stats(cleaned_df, 'date.weekDay')

# Change value of `date.weekDay` to meaningful name
WEEK_DAY = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
weekDay_stats['date.weekDay'] \
    = weekDay_stats['date.weekDay'].apply(lambda x: WEEK_DAY[x])

# Plot bar chart
# ====================  For `visits` ==================== 
fig = px.bar(
    weekDay_stats, x="date.weekDay", y="visits",
    color='visits', hover_name='visits',
    labels={'date.weekDay': 'DayOfWeek', 'visits': '#Visits'},
    color_continuous_scale=px.colors.sequential.Tealgrn,
    height=420, width=680
)
fig.layout.update(
    BASIC_PLOTLY_CONFIG |
    dict(title_text='<b>Visits by Day-of-week </b>',
         xaxis_title='Day of week', yaxis_title='# Visits')
)
iplot(fig)

# ====================  For `non_zero_transactions` ==================== 
fig = px.bar(
    weekDay_stats, x="date.weekDay", y="non_zero_transactions",
    color='non_zero_transactions', hover_name='non_zero_transactions',
    labels={'date.weekDay': 'DayOfWeek', 'non_zero_transactions': '#Non-zero transactions'},
    color_continuous_scale=px.colors.sequential.Viridis,
    height=420, width=680
)
fig.layout.update(
    BASIC_PLOTLY_CONFIG |
    dict(title_text='<b>Non-zero transactions by Day-of-week</b>',
         xaxis_title='Day of week', yaxis_title='# Non-zero transactions')
)
iplot(fig)

# ====================  For `totals.transactionRevenue` ====================
fig = px.bar(
    weekDay_stats, x="date.weekDay", y="totals.transactionRevenue",
    color='totals.transactionRevenue', hover_name='totals.transactionRevenue',
    labels={'date.weekDay': 'DayOfWeek', 'totals.transactionRevenue': 'ΣRevenue'},
    color_continuous_scale=px.colors.sequential.Peach,
    height=420, width=680
)
fig.layout.update(
    BASIC_PLOTLY_CONFIG |
    dict(title_text='<b>Transaction revenue by Day-of-week</b>',
         xaxis_title='Day of week', yaxis_title='Σ Revenue')
)
iplot(fig)

> 💭 <u>**Nhận xét**</u>: 
> * Nhìn vào ba biểu đồ ở trên, ta phát hiện một điểm chung đáng chú ý: các ngày trong tuần (từ thứ 2 $\to$ 6) thường có số lượng truy cập vào cửa hàng, số lượng phiên giao dịch và tổng doanh thu nhiều hơn so với thứ 7 và chủ nhật. 
>
> * Ta biết rằng phần lớn mọi người thường đi làm vào các ngày trong tuần và nghỉ vào cuối tuần, việc một lượng lớn khách hàng tiềm năng thường xuất hiện vào các ngày trong tuần có thể đến từ các khoảng thời gian nghỉ ngơi trong giờ làm việc. Khi này, người đi làm sẽ có chút ít thời gian rãnh rỗi để lên mạng xã hội, xem tin tức và ghé qua các cửa hàng để xem quần áo, v.v. Dẫn đến phần lớn doanh thu của cửa hàng đều xuất phát từ các ngày trong tuần. 
>
> * Còn vào các ngày cuối tuần, mọi thường có kế hoạch nghỉ ngơi, vui chơi riêng bên cạnh người thân, bạn bè nên họ thường không dành nhiều thời gian để lên mạng. Điều này có thể lý giải cho số lượng truy cập và tổng doanh thu vào thứ 7 và chủ nhật thường ít hơn hẳn các ngày trong tuần.

#### 3. Phân tích tổng quát xu hướng của người dùng trên toàn bộ tập dữ liệu

In [8]:
# Get some useful statistics from `date` column
date_stats = _create_feature_stats(cleaned_df, 'date')

# Plot bar chart
BASIC_LAYOUT_CONFIG = dict(
    title_x=0.5, title_font_size=24,
    yaxis_title_font_size=18, yaxis_tickfont_size=12,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=3, label='3m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    )
)

# ====================  For `visits` ====================
trace = go.Scatter(
    x=date_stats['date'].astype(str), y=date_stats['visits'],
    opacity=0.9, line=dict(color='#3fe0c0', shape='vh', width=2)
)
layout = dict(
    title="<b>Visits by date</b>", yaxis_title='# Visits', 
    **BASIC_LAYOUT_CONFIG
)
fig = dict(data=trace, layout=layout)
iplot(fig)

# ====================  For `non_zero_transactions` ====================
trace = go.Scatter(
    x=date_stats['date'].astype(str), y=date_stats['non_zero_transactions'],
    opacity=0.8, line=dict(color='#1068c9', shape='hvh', width=2)
)
layout = dict(
    title="<b>Non-zero transactions by date</b>", 
    yaxis_title='# Non-zero transactions',
    **BASIC_LAYOUT_CONFIG
)
fig = dict(data=trace, layout=layout)
iplot(fig)

# ====================  For `totals.transactionRevenue` ====================
trace = go.Scatter(
    x=date_stats['date'].astype(str), y=date_stats['totals.transactionRevenue'],
    opacity=0.8, line=dict(color='#ff666e', shape='spline', width=2)
)
layout = dict(
    title= "<b>Revenue by date</b>", yaxis_title='$ Revenue',
    **BASIC_LAYOUT_CONFIG
)
fig = dict(data=trace, layout=layout)
iplot(fig)

> 💭 <u>**Nhận xét**</u>: 
> * Trong phần tiền xử lý dữ liệu, ta biết được một thông tin là phần lớn người dùng truy cập đến GStore đều đến từ các quốc gia ở phương Tây. Từ kiến thức thực tế, ta đều biết các quốc gia phương Tây rất coi trọng các dịp lễ cuối năm như: Halloween, lễ Tạ ơn, Giáng sinh, năm mới, v.v. và người dân nước họ thường có các đợt nghỉ lễ kéo dài trong khoảng thời gian này. Và để chuẩn bị thật tốt cho dịp nghỉ lễ thì người dân thường có xu hướng mua sắm vật dụng cá nhân nhiều hơn.
>
> * Nhìn vào hai biểu đồ đầu tiên, ta cũng thấy rất rõ điều này. Ba tháng cuối năm (từ tháng 10 $\to$ 12) thường chứng kiến số lượng người truy cập tăng đáng kể và duy trì ở mức ổn định trong một khoảng thời gian liên tục. Con số này chỉ thực sự giảm xuống mức tiêu chuẩn khi gần kết thúc năm cũ, lúc này mọi người đều đã chuẩn bị đầy đủ vật dụng và tận hưởng kỳ nghỉ của mình nên họ không có nhu cầu mua sắm gì thêm.
>
> * Nhìn vào biểu đồ thứ hai, ta còn phát hiện một thông tin rất thú vị. Tiếp tục với câu chuyện ba tháng cuối năm, có vẻ như trong khoảng thời gian ban đầu từ tháng 10 đến gần cuối tháng 11 thì người dùng có thói quen lên cửa hàng CHỈ ĐỂ lựa chọn món hàng mình ưng ý chứ chưa quyết định bỏ tiền mua món đó. Chỉ khi bước vào giai đoạn chuyển tiếp từ tháng 11 sang mùa giáng sinh thì người dùng mới quyết định mua món hàng mà mình đã chọn trước. Thông qua biểu đồ, ta thấy khoảng thời gian từ đầu tháng 10 đến gần cuối tháng 11 thì số lượng giao dịch không thay đổi quá nhiều. Chỉ khi tháng 12 đến gần thì số lượng giao dịch mới tăng lên đáng kể. Và con số này trở về bình thường khi sắp đến giao thừa.
>
> * Vì dữ liệu doanh thu ban đầu của chúng ta có hơn *98%* giá trị bị thiếu, nên khi quan sát biểu đồ cuối cùng thì ta khó có thể nắm bắt được xu hướng cụ thể trong việc chi tiêu của khách hàng. Nhìn sơ qua dữ liệu của năm 2016 thì ta thấy tổng doanh thu của quý 4 có vẻ cao hơn quý 3 nhưng tương đối khó quan sát (và có phần gượng ép). Tuy nhiên ta vẫn nhìn thấy một xu hướng chung là người dùng sẽ sẵn sàng tiền vào các dịp lễ lớn như: giáng sinh, lễ tình nhân, v.v.
>
> * Đây là một thông tin rất hữu ích cho nhóm làm marketing, quảng cáo sản phẩm. Họ có thể đầu tư nhiều tiền hơn trong việc lên ý tưởng và tổ chức các chương trình khuyến mãi vào các dịp lễ lớn trong năm nói chung và đặc biệt là trong tháng 12 nói riêng. Biết được khách hàng thường mua sắm vào các dịp lễ cũng mở ra một thị trường tiềm năng cho các mặt hàng quà tặng theo mùa (socola vào Valentine), đây là cơ hội để các bên phát triển sản phẩm tận dụng hết khả năng của mình để thu được lợi nhuận từ khách hàng... Việc đầu tư phát triển sản phẩm và phát động chiến dịch quảng cáo vào những thời điểm thích hợp sẽ giúp cho những ông chủ thu được lợi nhuận cao nhất chỉ với chi phí thấp nhất.

---

# ---------- BREAK ----------

In [9]:
chapter break

SyntaxError: invalid syntax (2544851060.py, line 1)